In [345]:
import numpy as np
import read_data
import math

# Node Declaration

In [346]:
# Node Declaration

class Node:
    
    def __init__(self):
        self.splitAttribute=-1
        self.pred=-1
        self.positives=0
        self.negatives=0
        self.children={}
        self.median=-1
        self.isNumeric=False
        self.ChildrenCorrectlyClassified=0

In [347]:
def getMedian(data,splitAttribute):
        return np.median(data[:,splitAttribute])

# Growing Tree

In [348]:
def divideData(validData,validLabel,splitAttribute,isNumeric):
    distributeData={}
    distributeLabel={}
    instances=len(validData)
        
    if(isNumeric):
        numpyData=np.asarray(validData,dtype=np.float32)
        median=getMedian(numpyData,splitAttribute)
            
        for i in range(instances):
            key=-1
            if(validData[i][splitAttribute]<=median):
                key=0
            else:
                key=1
            if key in distributeData:
                distributeData[key].append(validData[i])
                distributeLabel[key].append(validLabel[i])
            
            else:
                distributeData[key]=[validData[i]]
                distributeLabel[key]=[validLabel[i]]

    else:
        for i in range(instances):
            key=validData[i][splitAttribute]
            
            if key in distributeData:
                distributeData[key].append(validData[i])
                distributeLabel[key].append(validLabel[i])
            
            else:
                distributeData[key]=[validData[i]]
                distributeLabel[key]=[validLabel[i]]
    return (distributeData,distributeLabel)
    

In [349]:
def growTree(data,label,hasBeenSplit,isNumeric):
    
    global nodes
    nodes+=1

#     get Best Splitting Attribute
    splitAttribute=getSplitAttribute(data,label,hasBeenSplit,isNumeric)
    
    instances=len(data)
    positives=0.0
    negatives=0.0
    
    for i in range(instances):
        if(label[i]==1):
            positives+=1
        else:
            negatives+=1
#     Declaring and Initialising Root
    root=Node()
    root.splitAttribute=splitAttribute
    root.positives=positives
    root.negatives=negatives
    if(positives>=negatives):
        root.pred=1
    else:
        root.pred=0
        
    if(splitAttribute==-1):
        return root
    if(positives==0 or negatives==0):
        return root
    
    hasBeenSplit[splitAttribute]=True

    if(isNumeric[splitAttribute]):
        root.isNumeric=True
        root.median=getMedian(np.asarray(data),splitAttribute)
        
    (distributeData,distributeLabel)=divideData(data,label,splitAttribute,root.isNumeric)
    
    if(len(distributeLabel)==1):
        root.splitAttribute=-1
        hasBeenSplit[splitAttribute]=False
        return root
    
    for key,value in distributeData.items():
        root.children[key]=growTree(value,distributeLabel[key],hasBeenSplit,isNumeric)
#         root.children[key].parent=root
        
    hasBeenSplit[splitAttribute]=False
    return root

# Selecting Best Attribute

In [386]:
def getSplitAttribute(data,label,hasBeenSplit,isNumeric):
    attributes=len(hasBeenSplit)
    
    curEntropy=Hy(label)
    
    bestAttribute=-1
    conditionalEntropy=1e9
    instances=len(data)
    numpyData=np.asarray(data,dtype=np.float32)
        
    for i in range(attributes):

        if(hasBeenSplit[i] and not isNumeric[i]):
            continue
        counterTable={}
        median=-1
        if(isNumeric[i]):
            median=getMedian(numpyData,i)
        
        for j in range(instances):
            key=-1
            if(not isNumeric[i]):
                key=data[j][i]
            else:
                if(data[j][i]<=median):
                    key=0
                else:
                    key=1
            
            if key in counterTable:
                counterTable[key][int(label[j])]+=1
            else:
                counterTable[key]=[0,0]
                counterTable[key][int(label[j])]+=1
        
        entropy=0
        for key,value in counterTable.items():
            prob=(value[0]+value[1])/float(instances)
            entropy+=prob*getEntropy(value[1],value[0])
        
        if(entropy<conditionalEntropy):
            conditionalEntropy=entropy
            bestAttribute=i
    
    informationGain=curEntropy-conditionalEntropy
    
#     print(informationGain)
    
#     if(informationGain<=0):
#         return -1
    
    return bestAttribute
            
                
        
        

In [351]:
def Hy(y):
    instances=len(y)
    positives=0
    negatives=0
    
    for i in range(instances):
        if(y[i]==0):
            negatives+=1
        else:
            positives+=1

    return getEntropy(positives,negatives)

In [352]:
def getEntropy(positives,negatives):
    p1=positives/float(positives+negatives)
    p0=negatives/float(positives+negatives)
    
    entropy=0
    
    if(negatives!=0):
        entropy+=(-p0*math.log(p0))
    
    if(positives!=0):
        entropy+=(-p1*math.log(p1))
    
    return entropy

## Count Nodes

In [406]:
def countNodes(root):
    if(root.splitAttribute==-1):
        return 1
    ans=1
    for key,values in root.children.items():
        ans+=countNodes(values)
    return ans

# Prediction & Accuracy

In [353]:
def getPrediction(root,instance):
    if(root.splitAttribute==-1):
        return root.pred
    
    key=-1
    
    if(root.isNumeric):
        if(instance[root.splitAttribute]<=root.median):
            key=0
        else:
            key=1
    else:
        key=instance[root.splitAttribute]
    
    if key in root.children:
        return getPrediction(root.children[key],instance)
    else:
        return root.pred



In [354]:
def getAccuracy(root,data,label):
    
    instance=len(data)
    correct=0
    
    for i in range(instance):
        if(label[i]==getPrediction(root,data[i])):
            correct+=1
    
    return correct/float(instance)

In [355]:
getEntropy(1,4)

0.5004024235381879

In [356]:
# test_data

In [357]:
# b=np.array([1,2,3])
# b.shape
# type((b.tolist())[0])

# Pruning

In [358]:
# def prune(root,originalRoot,validData,validLabel):
    
#     bestAccuracy=0
#     bestParent=root
#     bestKey=-1
    
#     temp=dict(root.children)
    
#     for key,value in temp.items():
# #         print(key)
#         root.children.pop(key, None)
#         accuracy=getAccuracy(originalRoot,validData,validLabel)
#         if(accuracy>bestAccuracy):
#             bestAccuracy=accuracy
#             bestKey=key
#         root.children[key]=value
    
#     for key,value in root.children.items():
#         tupleA=prune(value,originalRoot,validData,validLabel)
#         if(tupleA[0]>bestAccuracy):
#             bestAccuracy=tupleA[0]
#             bestParent=tupleA[1]
#             bestKey=tupleA[2]
    
#     return (bestAccuracy,bestParent,bestKey)
        

In [359]:
# def efficientPrune(root,originalRoot,validData,validLabel,firstTime):
    
#     bestAccuracy=0
#     bestParent=root
#     bestKey=-1
    
#     temp=dict(root.children)
    
#     if(not firstTime and root.recalculate==False):
#         return root.prunedTuple

#     for key,value in temp.items():
#         root.children.pop(key, None)
#         accuracy=getAccuracy(originalRoot,validData,validLabel)
#         if(accuracy>bestAccuracy):
#             bestAccuracy=accuracy
#             bestKey=key
#         root.children[key]=value
    
#     for key,value in root.children.items():
#         tupleA=efficientPrune(value,originalRoot,validData,validLabel,firstTime)
#         if(tupleA[0]>bestAccuracy):
#             bestAccuracy=tupleA[0]
#             bestParent=tupleA[1]
#             bestKey=tupleA[2]
    
#     root.recalculate=False
#     root.prunedTuple=(bestAccuracy,bestParent,bestKey)
#     return (bestAccuracy,bestParent,bestKey)
        

In [360]:
# def updateTillParent(root):
    
#     while root is not None:
#         root.recalculate=True
#         root=root.parent
        

# Run Slow Pruning

In [361]:
# formerBest=getAccuracy(root,validX,validY)
# while True:
#     print ("Hi")
#     (bestAccuracy,pruneNodeParent,prunedKey)=prune(root,root,validX,validY)
#     print(formerBest," ",bestAccuracy)
#     if(bestAccuracy>formerBest):
#         pruneNodeParent.children.pop(prunedKey, None)
#         formerBest=bestAccuracy
#     else:
#         break

In [362]:
# formerBest=getAccuracy(root,validX,validY)
# firstTime=True
# while True:
#     print ("Hi")
#     (bestAccuracy,pruneNodeParent,prunedKey)=efficientPrune(root,root,validX,validY,firstTime)
#     firstTime=False
#     print(formerBest," ",bestAccuracy)
#     if(bestAccuracy>formerBest):
#         pruneNodeParent.children.pop(prunedKey, None)
#         formerBest=bestAccuracy
#         updateTillParent(pruneNodeParent)
#     else:
#         break

In [363]:
# https://leemendelowitz.github.io/blog/how-does-python-find-packages.html

# Efficient Pruning

In [364]:
def newprune(root,validData,validLabel):
    
    bestIncrease=0
    bestNode=root

    if(root.splitAttribute==-1):
        return (bestIncrease,bestNode)
    
    pruneCorrect=0
    instances=len(validData)
    for i in range(instances):
        if(validLabel[i]==root.pred):
            pruneCorrect+=1
    
    if(bestIncrease<(pruneCorrect-root.ChildrenCorrectlyClassified)):
        bestIncrease=(pruneCorrect-root.ChildrenCorrectlyClassified)
        
    (distributeData,distributeLabel)=divideData(validData,validLabel,root.splitAttribute,root.isNumeric)
    
    for key,value in root.children.items():
        if key not in distributeData:
            continue
        tupleA=newprune(value,distributeData[key],distributeLabel[key])
        if(tupleA[0]>bestIncrease):
            bestIncrease=tupleA[0]
            bestNode=tupleA[1]
    
    return (bestIncrease,bestNode)

In [365]:
def prePrune(root,validData,validLabel):
    
    instances=len(validData)
    correctlyDone=0
    if(root.splitAttribute==-1):
        for i in range(instances):
            if(validLabel[i]==root.pred):
                correctlyDone+=1
    
    else:
        (distributeData,distributeLabel)=divideData(validData,validLabel,root.splitAttribute,root.isNumeric)
        
        for key,value in distributeData.items():
            if key in root.children:
                correctlyDone+=prePrune(root.children[key],value,distributeLabel[key])
            else:
                temp=len(value)
                for i in range(temp):
                    if(distributeLabel[key][i]==root.pred):
                        correctlyDone+=1
    
    root.ChildrenCorrectlyClassified=correctlyDone
    return correctlyDone

# Testing Code Training and Pruning (Part a and b)

In [389]:
train_data = read_data.preprocess("Dataset/DT/dtree_data/train.csv")
valid_data = read_data.preprocess("Dataset/DT/dtree_data/valid.csv")
test_data = read_data.preprocess("Dataset/DT/dtree_data/test.csv")

In [390]:
# train_data = read_data.preprocess("Dataset/DT/dtree_data/train.csv",False)
# valid_data = read_data.preprocess("Dataset/DT/dtree_data/valid.csv",False)
# test_data = read_data.preprocess("Dataset/DT/dtree_data/test.csv",False)

In [391]:
print (train_data.shape)
print (valid_data.shape)
print (test_data.shape)
type(train_data.shape)

(27000, 15)
(3000, 15)
(7000, 15)


tuple

In [392]:
def getLists(train_data):
    (instances,attributes)=train_data.shape
    trainLabel=train_data[:,0]
    trainData=train_data[:,1:attributes]
    trainDataList=trainData.tolist()
    trainLabelList=trainLabel.tolist()
    return(trainDataList,trainLabelList)


In [393]:
(trainX,trainY)=getLists(train_data)
(validX,validY)=getLists(valid_data)
(testX,testY)  =getLists(test_data)

In [394]:
hasBeenSplit=[]
isNumeric=[]
attributes=train_data.shape[1]
for i in range(attributes-1):
    hasBeenSplit.append(False)
    isNumeric.append(False)

# isNumeric[0]=True
# isNumeric[2]=True
# isNumeric[4]=True
# isNumeric[10]=True
# isNumeric[11]=True
# isNumeric[12]=True

In [395]:
# print (validX[0])
# print(validX[1])
# print(testX[0])

In [400]:
nodes=0
root=growTree(trainX,trainY,hasBeenSplit,isNumeric)

In [401]:
print (nodes)
print (getAccuracy(root,trainX,trainY))
print (getAccuracy(root,testX,testY))
print (getAccuracy(root,validX,validY))

7962
0.8893333333333333
0.8024285714285714
0.7916666666666666


In [404]:
pruned=0
while True:
    val=prePrune(root,validX,validY)
    print ("PrePruned")
    (bestIncrease,bestNode)=newprune(root,validX,validY)
    print(bestIncrease)
    if(bestIncrease>0):
        pruned+=1
        bestNode.splitAttribute=-1
    else:
        break

PrePruned
0


In [407]:
print (pruned)
print (countNodes(root))
print (getAccuracy(root,trainX,trainY))
print (getAccuracy(root,testX,testY))
print (getAccuracy(root,validX,validY))

0
2378
0.8507777777777777
0.8232857142857143
0.835
